# Facial Emotion Recognition with PyTorch

This notebook demonstrates real-time facial emotion recognition using a PyTorch model. The application captures video from a webcam, detects faces, and predicts the emotion displayed by each face.

## Requirements

- Python 3.6+
- PyTorch
- OpenCV (cv2)
- NumPy
- A webcam


## Import Libraries

In [ ]:
import cv2
import numpy as np
import torch
import os
import time

## Load the PyTorch Model

In [ ]:
# Check if CUDA is available, otherwise use CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Path to the model file
model_path = 'models/model.pth'

# Check if model file exists
if not os.path.exists(model_path):
    raise FileNotFoundError(f'Model file not found at {model_path}')

# Load the model
try:
    model = torch.load(model_path, map_location=device)
    model.to(device)
    model.eval()  # Set the model to evaluation mode
    print('Model loaded successfully')
except Exception as e:
    print(f'Error loading model: {e}')

## Define Emotion Labels

In [ ]:
# Define emotion labels (ensure they match the order used in training)
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']
print(f'Emotion classes: {emotion_labels}')

## Real-time Emotion Detection Function

In [ ]:
def detect_emotions():
    # Initialize webcam
    cap = cv2.VideoCapture(0)
    
    # Check if webcam opened successfully
    if not cap.isOpened():
        print('Error: Could not open webcam')
        return
    
    # Load face detection model
    try:
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    except Exception as e:
        print(f'Error loading face cascade classifier: {e}')
        cap.release()
        return
    
    print('Starting emotion detection. Press q to exit.')
    
    # FPS calculation variables
    frame_count = 0
    start_time = time.time()
    fps = 0
    
    while True:
        # Read frame from webcam
        ret, frame = cap.read()
        if not ret:
            print('Error: Failed to capture frame')
            break
        
        # Mirror the frame for more intuitive display
        mirrored_frame = cv2.flip(frame, 1)
        
        # Convert to grayscale for face detection
        gray = cv2.cvtColor(mirrored_frame, cv2.COLOR_BGR2GRAY)
        
        # Detect faces
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)
        
        # Process each detected face
        for (x, y, w, h) in faces:
            # Extract face region
            face_roi = gray[y:y + h, x:x + w]
            
            # Preprocess for PyTorch model
            face_resized = cv2.resize(face_roi, (48, 48))
            
            # Convert to PyTorch tensor and normalize
            face_tensor = torch.from_numpy(face_resized).float() / 255.0
            face_tensor = (face_tensor - 0.5) / 0.5  # Normalize to [-1, 1]
            face_tensor = face_tensor.unsqueeze(0).unsqueeze(0)  # Add batch and channel dimensions
            face_tensor = face_tensor.to(device)
            
            # Make prediction
            try:
                with torch.no_grad():
                    prediction = model(face_tensor)
                    emotion_index = torch.argmax(prediction, dim=1).item()
                    emotion = emotion_labels[emotion_index]
                    
                # Draw rectangle around face and display emotion
                cv2.rectangle(mirrored_frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
                cv2.putText(mirrored_frame, emotion, (x, y - 10), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)
            except Exception as e:
                print(f'Error during prediction: {e}')
        
        # Calculate and display FPS
        frame_count += 1
        elapsed_time = time.time() - start_time
        if elapsed_time > 1.0:  # Update FPS every second
            fps = frame_count / elapsed_time
            frame_count = 0
            start_time = time.time()
        
        # Display FPS on frame
        cv2.putText(mirrored_frame, f'FPS: {fps:.2f}', (10, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
        # Display the frame
        cv2.imshow("Emotion Detection (Mirrored)", mirrored_frame)
        
        # Press 'q' to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Release resources
    cap.release()
    cv2.destroyAllWindows()
    print('Emotion detection stopped')

## Run the Emotion Detection

In [ ]:
# Run the emotion detection
detect_emotions()